In [ ]:
#@title Initialize environment { vertical-output: true, display-mode: "form" }
import os
import subprocess
import random
import time

import glob

try:
    import fiona
except ImportError:
    print('fiona package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'fiona'])
    import fiona

try:
    import rasterio.merge
except ImportError:
    print('rasterio package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'rasterio'])
    import rasterio.merge

fiona package not installed. Installing ...
rasterio package not installed. Installing ...


In [ ]:
#@title Initialize Google Drive  environments { vertical-output: true, display-mode: "form" }
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
#@title Initialize GEE environments { vertical-output: true, display-mode: "form" }
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=SUuHjH2x6cOMLdrOGmEEDqK_G8DDJLEagB_1m0qcWBc&tc=CE2RKKGsDEhDR3fX8e9MBXZjalNGyWnc5YuIS61o6NE&cc=O8xn8m44-A4fRV967PRBJcz9sYW9-OxsPu9g4JleEI4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvuBDCITnhwR70NJaUTTocrjfdmIXlr_cU95DRUs2kc1ciShH5S0K-4

Successfully saved authorization token.
geemap package not installed. Installing ...


In [ ]:
# !rm grid.gpkg

In [ ]:
#@title Import vector layer wich contained border areas for export  { vertical-output: true, display-mode: "form" }
#@markdown Path on googledrive and file name
folder_path = "GeoData/InstaMaps/Portugal/" #@param {type:"string"}
grid_file_name =  "portugal_50km_grid"#@param {type:"string"}

grid_file = grid_file_name+'.gpkg' # grid file

subprocess.check_call(['cp', '/content/gdrive/MyDrive/'+folder_path+grid_file, '.'])

os.rename(grid_file, 'grid.gpkg')
!ls

gdrive	grid.gpkg  sample_data


In [ ]:
#@title Output settings  { vertical-output: true, display-mode: "form" }
#@markdown Path on googledrive and scale
export_folder = "GEE_data_export_Portugal" #@param {type:"string"}
scale =  10#@param {type:"integer"}
crs_epsg = "3857" #@param {type:"string"}

In [ ]:
#@title Convert grid to ee.FeatureCollection  { vertical-output: true, display-mode: "form" }
polygon_features = []
with fiona.open('grid.gpkg') as layer:
    for feature in layer:
        polygon_features.append(ee.Feature(feature))
AOI = ee.FeatureCollection(polygon_features)

In [ ]:
#@title Import <b>ESA WorldCover 10m v200</b>  { vertical-output: true, display-mode: "form" }
landcover = ee.ImageCollection('ESA/WorldCover/v200').first();

In [ ]:
#@title Generation and export tiles  { vertical-output: true, display-mode: "form" }
%%time
for poly in polygon_features:
#    if int(poly.getInfo()["id"]) == 2186:
        Feature_polygon = ee.Geometry(poly.geometry())
        ee.batch.Export.image.toDrive(image = landcover, region=Feature_polygon, folder=export_folder,
        description = 'ESA_WorldCover_50km_'+poly.getInfo()["id"], crs='EPSG:3857', scale = scale, maxPixels = 1e10,fileFormat = 'GeoTIFF').start()
        print(poly.getInfo()["id"])
        time.sleep(round(random.uniform(5, 20)))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
CPU times: user 10.5 s, sys: 1.03 s, total: 11.6 s
Wall time: 19min 22s
